In [3]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import pandas as pd
import glob

from sklearn.metrics import precision_score, recall_score

In [4]:
# 이미지 경로 설정
image_paths = glob.glob('output_folder/*.jpg')

In [12]:
# 색상 공간 변환 및 특정 채널 조합 탐지 함수
def extract_channels(image, color_space='HSV', use_channels='all'):
    if color_space == 'HSV':
        hsv_image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
        h, s, v = cv.split(hsv_image)
        if use_channels == 'all': return h, s, v
        elif use_channels == 'H': return h, None, None
        elif use_channels == 'SV': return None, s, v
    elif color_space == 'YUV':
        yuv_image = cv.cvtColor(image, cv.COLOR_BGR2YUV)
        y, u, v = cv.split(yuv_image)
        if use_channels == 'all': return y, u, v
        elif use_channels == 'UV': return None, u, v
        elif use_channels == 'Y': return y, None, None
    elif color_space == 'YCbCr':
        ycbcr_image = cv.cvtColor(image, cv.COLOR_BGR2YCrCb)
        y, cb, cr = cv.split(ycbcr_image)
        if use_channels == 'all': return y, cb, cr
        elif use_channels == 'CbCr': return None, cb, cr
        elif use_channels == 'Y': return y, None, None
    else:
        raise ValueError("Invalid color space or channels selection.")

In [13]:
# 보라색 원 탐지 함수
def detect_purple_mask(image, color_space='HSV', use_channels='all'):
    if color_space == 'HSV' and use_channels == 'SV':
        _, s, v = extract_channels(image, 'HSV', 'SV')
        merged_channels = [ch for ch in [s, v] if ch is not None]
        lower_purple, upper_purple = (50, 50), (255, 255)
        mask = cv.inRange(cv.merge(merged_channels), lower_purple, upper_purple)

    elif color_space == 'YUV' and use_channels == 'UV':
        _, u, v = extract_channels(image, 'YUV', 'UV')
        merged_channels = [ch for ch in [u, v] if ch is not None]
        lower_purple, upper_purple = (100, 120), (200, 140)
        mask = cv.inRange(cv.merge(merged_channels), lower_purple, upper_purple)
        
    elif color_space == 'YCbCr' and use_channels == 'CbCr':
        _, cb, cr = extract_channels(image, 'YCbCr', 'CbCr')
        merged_channels = [ch for ch in [cb, cr] if ch is not None]
        lower_purple, upper_purple = (100, 120), (200, 140)
        mask = cv.inRange(cv.merge(merged_channels), lower_purple, upper_purple)

    else:
        raise ValueError("Invalid color space or channel combination.")
    
    result = cv.bitwise_and(image, image, mask=mask)
    return mask, result

In [14]:
# 시각화 및 탐지 성능 비교
def visualize_and_compare(image_paths):
    results = []
    color_spaces = [('HSV', 'all'), ('HSV', 'SV'), ('YUV', 'UV'), ('YUV', 'Y'), ('YCbCr', 'CbCr')]
    
    for image_path in image_paths:
        img = cv.imread(image_path)
        
        for color_space, use_channels in color_spaces:
            mask, result = detect_purple_mask(img, color_space=color_space, use_channels=use_channels)
            purple_pixels = np.sum(mask > 0)  # 탐지된 보라색 픽셀 개수

            # 결과 저장
            results.append({
                'Image': image_path,
                'Color Space': color_space,
                'Channel': use_channels,
                'Detected Pixels': purple_pixels
            })
            
            # 각 조합별 결과 시각화
            plt.figure(figsize=(8, 4))
            plt.subplot(1, 2, 1)
            plt.imshow(mask, cmap='gray')
            plt.title(f"{color_space}-{use_channels} Mask")
            plt.axis('off')
            
            plt.subplot(1, 2, 2)
            plt.imshow(cv.cvtColor(result, cv.COLOR_BGR2RGB))
            plt.title(f"{color_space}-{use_channels} Result")
            plt.axis('off')
            plt.show()
    
    # 결과 비교 분석
    results_df = pd.DataFrame(results)
    sns.boxplot(x='Color Space', y='Detected Pixels', hue='Channel', data=results_df)
    plt.title('Purple 색상 범위 구하기 (by pixel color/channel)')
    plt.show()

    return results_df

In [15]:
# 탐지 결과를 바탕으로 가장 성능이 좋은 조합을 찾고 테스트 이미지에 적용
def apply_best_combination(results_df, test_images):
    best_combination = results_df.groupby(['Color', 'Channel'])['Detected Pixels'].mean().idxmax()
    color_space, use_channels = best_combination
    
    print(f"최적의 색상값 : {color_space}-{use_channels}")
    
    for test_image_path in test_images:
        test_img = cv.imread(test_image_path)
        mask, result = detect_purple_mask(test_img, color_space=color_space, use_channels=use_channels)
        
        # 테스트 이미지 결과 시각화
        plt.figure(figsize=(8, 4))
        plt.subplot(1, 2, 1)
        plt.imshow(mask, cmap='gray')
        plt.title(f"최적의 색상값 ({color_space}-{use_channels})")
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.imshow(cv.cvtColor(result, cv.COLOR_BGR2RGB))
        plt.title(f"최적의 색상값 ({color_space}-{use_channels})")
        plt.axis('off')
        plt.show()

# 테스트
# 랜덤으로 5개 이미지 선택하여 성능 분석
random_image_paths = random.sample(image_paths, 5)
results_df = visualize_and_compare(random_image_paths)

# 테스트 이미지에 가장 좋은 조합 적용
test_images = ['test_image1.jpg', 'test_image2.jpg']  # 테스트 이미지 경로 예시
apply_best_combination(results_df, test_images)

ValueError: Invalid color space or channel combination.